In [1]:
# Import the SPICE module
import spiceypy

In [2]:
# We want to determine the position of our home planet with respect to the Sun.
# The datetime shall be set as "today" (midnight). SPICE requires the Ephemeris Time (ET);
# thus, we need to convert a UTC datetime string to ET.

import datetime

# get today's date
DATE_TODAY = datetime.datetime.today().strftime('%Y-%m-%dT00:00:00')

# convert the utc midnight string to the corresponding ET
ET_TODAY_MIDNIGHT = spiceypy.utc2et(DATE_TODAY)

SpiceNOLEAPSECONDS: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(NOLEAPSECONDS) --

The variable that points to the leapseconds (DELTET/DELTA_AT) could not be located in the kernel pool.  It is likely that the leapseconds kernel has not been loaded.

utc2et_c --> UTC2ET --> TTRANS

================================================================================

In [3]:
# oh... an error occurred. The error tells us that a so called "kernel" is missing. These kernels store all
# information that are required for time conversion, pointing, position determination etc.
# For this tutorial the Git repository contains already the necessary kernel. We need to load it first
spiceypy.furnsh('../_kernels/lsk/naif0012.tls')

In [4]:
# Let's re-try our first time conversion command
ET_TODAY_MIDNIGHT = spiceypy.utc2et(DATE_TODAY)
print(ET_TODAY_MIDNIGHT)

753710469.182823


In [5]:
# Can we compute now the position and velocity (so called state) of the Earth with respect to the Sun?
# We use the following function to determine the state vector and the so called light time (travel time of the
# light between the Sun and our home planet). Positions are always given in km, velocities in km/s and times in
# seconds

# targ : Object that shall be pointed at
# et : The ET of the computation
# ref : The reference frame. Here, it is ECLIPJ2000 (so Medium article)
# obs :  The observer respectively the center of our state vector computation
sun_earth, sun_earth_lt = spiceypy.spkgeo(targ=399, et=ET_TODAY_MIDNIGHT, ref='ECLIPJ2000', obs=10)

SpiceNOLOADEDFILES: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(NOLOADEDFILES) --

At least one SPK file needs to be loaded by SPKLEF before beginning a search.

spkgeo_c --> SPKGEO --> SPKSFS

================================================================================

In [6]:
# An error occured. Again a kernel error. Well, we need to load a so called spk to load positional information:
spiceypy.furnsh('../_kernels/spk/de432s.bsp')

In [7]:
sun_earth, sun_earth_lt = spiceypy.spkgeo(targ=399, et=ET_TODAY_MIDNIGHT, ref='ECLIPJ2000', obs=10)

In [8]:
# The state vector is 6 dimensional: x,y,z in km and the corresponding velocities in km/s
print('State vector of the Earth w.r.t. the Sun for "today" (midnight):\n', sun_earth)

State vector of the Earth w.r.t. the Sun for "today" (midnight):
 [ 8.03805363e+07  1.24080074e+08 -7.03550232e+03 -2.54936320e+01
  1.60749310e+01 -1.26653757e-03]


In [9]:
# The (Euclidean) distance should be around 1 AU. Why "around"? Well the Earth revolves the Sun in a slightly
# non-perfect circle (elliptic orbit). First, we compute the distance in km.
import numpy as np
sun_earth_dist = np.linalg.norm(sun_earth[:3])

# Convert the distance in astronomical units (1 AU)
# Instead of searching for the "most recent" value, we use the default value in SPICE. This way, we can easily
# compare our results with the results of others.
sun_earth_dist_au = spiceypy.convrt(sun_earth_dist, 'km', 'AU')

print('Current distance between the Earth and the Sun in AU:', sun_earth_dist_au)

Current distance between the Earth and the Sun in AU: 0.98825455576592


# Orbital speed of the Earth
For this, we need the equation to determine the orbital speed. We assume that the Sun's mass is greater than the mass
of the Earth and we assume that our planet is moving on an almost circular orbit. The orbit velocity $v_{\text{orb}}$ can be approximated with, where $G$ is the gravitational constant, $M$ is the mass of the Sun and $r$ is the distance between the Earth and the Sun:
\begin{align}
    v_{\text{orb}}\approx\sqrt{\frac{GM}{r}}
\end{align}

In [10]:
# First, we compute the actual orbital speed of the Earth around the Sun
sun_earth_speed = np.linalg.norm(sun_earth[3:])

# It's around 30 km/s
print('Current orbital speed of the Earth around the Sun in km/s:', sun_earth_speed)

Current orbital speed of the Earth around the Sun in km/s: 30.13849166515453


In [11]:
# Now let's compute the theoretical expectation. First, we load a pck file that contain miscellanoeus information,
# like the G*M values for different objects

# First, load the kernel
spiceypy.furnsh('../_kernels/pck/gm_de431.tpc')
_, GM_SUN = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)

# Now compute the orbital speed
v_orb = lambda gm, r: np.sqrt(gm/r)
sun_earth_speed_theory = v_orb(GM_SUN[0], sun_earth_dist)

# Print the result
print('Theoretical orbital speed of the Earth around the Sun in km/s:', sun_earth_speed_theory)

Theoretical orbital speed of the Earth around the Sun in km/s: 29.96116516023395
